In [1]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [40]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

# import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality']

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

In [41]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# optR = OptimizedRounder()
# optR.fit(regression_predictions, original_labels)
# print(optR.coefficients)

# XGBoost Regression

In [42]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
# for i in range(100):
    
#     print(i)
    ## Building RF model
XGB_md = XGBRegressor(tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 5.5, 
                           learning_rate = 0.031, 
                           max_depth = 5, 
                           min_child_weight = 68, 
                           n_estimators = 8800, 
                           subsample = 0.41, 
                           random_state = 42).fit(X, Y)
XGB_imp.append(XGB_md.feature_importances_)
    
## Predicting on X_test and test
XGB_pred_1 = XGB_md.predict(X)
XGB_pred_2 = XGB_md.predict(test_md)

optR = OptimizedRounder()
optR.fit(XGB_pred_1, Y)
coef = optR.coefficients()
XGB_pred_1 = optR.predict(XGB_pred_1, coef).astype(int)
XGB_pred_2 = optR.predict(XGB_pred_2, coef).astype(int)

# XGB_pred_2 = optR.predict(XGB_pred_1, Y)

# print(optR.coefficients)
    
# ## Computing roc-auc score
XGB_cv_scores.append(cohen_kappa_score(Y, XGB_pred_1, weights = 'quadratic'))
# preds.append(XGB_pred_2)

# XGB_cv_score = np.mean(XGB_cv_scores)    
# print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

In [43]:
XGB_cv_scores

[0.5677507025719469]

In [45]:
XGB_pred_2.min()

5

In [35]:
optR.coefficients()

array([2.86540979, 4.74084782, 5.76910717, 6.60348916, 7.64214326])

In [29]:
XGB_pred_1

array([5, 5, 5, ..., 5, 3, 5])

In [46]:
pd.DataFrame({'Y': Y, 'Y_pred': XGB_pred_1})

,Y,Y_pred
0,6,7
1,6,7
2,7,7
3,5,5
4,6,6
...,...,...
2051,7,7
2052,5,5
2053,8,7
2054,5,5


In [47]:
pd.crosstab(Y, XGB_pred_1)

col_0,5,6,7
quality,,,
3,11,1,0
4,39,15,1
5,632,180,27
6,230,366,182
7,34,113,186
8,0,11,28


In [24]:
XGB_cv_scores

[0.3261688599539273]

In [22]:
optR.predict(XGB_pred_1, Y)

array([5., 5., 5., ..., 5., 3., 5.], dtype=float32)

In [15]:
XGB_pred_1

array([3.7313   , 3.3256998, 3.2479517, ..., 3.7407355, 2.0769548,
       3.4908228], dtype=float32)

In [18]:
optR.coefficients

<bound method OptimizedRounder.coefficients of <__main__.OptimizedRounder object at 0x7f63573de830>>

In [14]:
print(optR.coefficients)

<bound method OptimizedRounder.coefficients of <__main__.OptimizedRounder object at 0x7f635734aa40>>
